# 重新构建数据生成流程 - 批量逻辑
1. 问题生成流程
2. 答案生成流程

具体流程：
0. 生成所有文章的chunk，并保存chunk数据集, chunk,chunk_id,source ,doc_id
1. 先批量生成问题，问题评估是否合格
2. 再批量生成答案，答案评估是否合格
3. chunk数据集构建 chunk,chunk_id,source ,doc_id
4. 问题数据集构建 [{chunk_*4, chunk_id, source}], question
5. 答案数据集构建 [{chunk_*4+chunk_d4, chunk_id, source}], question, answer
6. 问题生成数据集 [{chunk_*4, chunk_id, source}] -> question1, question2
6. 答案生成数据集 [{chunk_*4, chunk_id, source}], question -> content, reasoning_content
7. 构建sft数据集 [{chunk_*4+chunk_d4, chunk_id, source}], question -> content, reasoning_content

## 生成所有文章的chunk

In [2]:
from utils.article_chunks import gen_chunks
article_chunks = gen_chunks("data", "outputs_chunks/article_chunks02.json")

d:\Miniconda3\envs\raft\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


len(file_names): 277


Chunking: 100%|██████████| 1/1 [00:00<00:00, 44.56file/s, chunks=34]


Chunks saved to outputs_chunks/article_chunks02.json


Chunking: 100%|██████████| 1/1 [00:00<00:00, 69.96file/s, chunks=26]


...

Chunks saved to outputs_chunks/article_chunks02.json


Chunking: 100%|██████████| 1/1 [00:00<00:00, 111.11file/s, chunks=24]


Chunks saved to outputs_chunks/article_chunks02.json


Chunking: 100%|██████████| 1/1 [00:00<00:00, 333.46file/s, chunks=25]

Chunks saved to outputs_chunks/article_chunks02.json
len(chunks_all) = 146


In [ ]:
from utils.article_chunks import read_file_names, build_or_load_chunks
data_dir = "data"
filenames = read_file_names(data_dir, ext=".md")
filenames[2]

len(file_names): 277


'10大新显示技术_llm_correct.md'

In [ ]:
import os
for filename in filenames:
    data_path = os.path.join(data_dir, filename)
    chunks,filename = build_or_load_chunks(data_path, "md")

In [ ]:
import os
from utils.article_chunks import read_file_names, build_or_load_chunks
from pathlib import Path
def get_article_chunks(data_dir):
    filenames = read_file_names(data_dir, ext=".md")
    filenames = filenames[:5]
    articles = []
    count = 0
    for idx,filename in enumerate(filenames):
        data_path = os.path.join(data_dir, filename)
        data_path = Path(data_path)
        chunks_dict, filename = build_or_load_chunks(data_path, "md", 512)
        article = {
            "doc_id": idx,
            "filename": filename,
            "chunks": chunks_dict
        }
        articles.append(article)
        count += len(chunks_dict)
    print(f"len(chunks_all) = {count}")
    return articles
    

d:\Miniconda3\envs\raft\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
articles = get_article_chunks("data")

len(file_names): 277


Chunking: 100%|██████████| 1/1 [00:00<00:00, 1000.79file/s, chunks=25]

len(chunks_all) = 146


[{'doc_id': 0,
  'filename': '1,8-萘酰亚胺类有机小分子电致发光材料的研究进展_王启_llm_correct.md',
  'chunks': [{'chunk': "# 1'8－萘酰亚胺类有机小分子电致发光材料的研究进展\n王 启 蒋 伟 王小亮 徐文连（东南大学化学化工学院，江苏南京210096）\n摘要 综述了近年来1'8－萘酰亚胺类小分子电致发光材料的研究进展，作了结构特点和相关理论的说明。关键词 萘酰亚胺 电致发光\n# Progress in 1,8－Naphthalimides as Organic Small Molecular Electroluminescent Materials\nWang Qi Jiang Wei Wang Xiaoliang Xu Wenlian（College of Chemistry and Chemical Engineering, Southeast University, Jiangsu Nanjing 210096）",
    'source': '1,8-萘酰亚胺类有机小分子电致发光材料的研究进展_王启_llm_correct.md',
    'chunk_id': 0},
   {'chunk': 'Abstract The research progress of 1,8-naphthalimide as organic small molecular electroluminescent materials was reviewed and the structure features and related theory were introduced.\nKeywords 1,8-naphthalimide electroluminescent',
    'source': '1,8-萘酰亚胺类有机小分子电致发光材料的研究进展_王启_llm_correct.md',
    'chunk_id': 1},
   {'chunk': '1,8－萘酰亚胺类化合物是一类荧光材料，是很重要的功能染料的中间体，广泛用于染料、荧光增白剂、荧光涂料等方面［1～5］。由于具有良好的光化学和热稳定性及结构易修饰等特点，近年来被研究用作高灵敏度荧光标记物和对某些特定金属离子具有响应

In [ ]:
from utils.article_chunks import get_article_chunks
article_chunks = get_article_chunks("data", "outputs_chunks/article_chunks01.json")

d:\Miniconda3\envs\raft\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


len(file_names): 277


Chunking: 100%|██████████| 1/1 [00:00<00:00, 500.10file/s, chunks=25]

len(chunks_all) = 146
Chunks saved to outputs_chunks/article_chunks01.json


In [ ]:
import json
def load_articles(chunks_path):
    with open(chunks_path, "r", encoding="utf-8") as f:
        articles = json.load(f)
        return articles

In [ ]:
def save_questions(questions, article_name, filename):
    # 判断 filename 是否存在，如果存在则追加写入，否则创建新文件
    if os.path.exists(filename):
        with open(filename, 'r', encoding="utf-8") as f:
            existing_questions = json.load(f)
        # 检查 article_name 是否已经存在于 questions 中
        if article_name in existing_questions:
            existing_questions[article_name].extend(questions)
        else:
            existing_questions[article_name] = questions
        with open(filename, 'w', encoding="utf-8") as f:
            json.dump(existing_questions, f, ensure_ascii=False, indent=4)
    else:
        with open(filename, 'w', encoding="utf-8") as f:
            json.dump({article_name: questions}, f, ensure_ascii=False, indent=4)
    print(f"Questions saved to {filename}")

In [ ]:
from utils.query_generation import get_chunkstr, generate_instructions_gen, build_openai_client_chat, get_chunk4
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

chat_model = build_openai_client_chat()

def query_generation(chunks_path):
    articles = load_articles(chunks_path)
    for article in articles:
        a_chunks = article["chunks"]
        a_name = article["filename"]
        futures = []
        # a_chunks 除 4 上取整数
        num_chunks = (len(a_chunks) // 4) + 1
        with tqdm(total=num_chunks, desc="Chunking", unit="file") as pbar:
            with ThreadPoolExecutor(max_workers=2) as executor:
                for i in range(0, len(a_chunks), 4):
                    chunk4 = get_chunk4(i, a_chunks)
                    # questions = generate_instructions_gen(chat_model, chunk4, x=2, model=" deepseek-r1-250120", prompt_key="deepseek-v2")
                    futures.append(executor.submit(generate_instructions_gen, chat_model, chunk4, x=2, model=" deepseek-r1-250120", prompt_key="deepseek-v2"))
                for future in as_completed(futures):
                    questions = future.result()
                    save_questions(questions, a_name, "outputs_questions/article_questions01.json")
                print(f"done {a_name} questions.")

In [ ]:
query_generation("outputs_chunks/article_chunks01.json")

In [1]:
from utils.query_generation import build_openai_client_chat, gen_query
chunks_path = "outputs_chunks/article_chunks01.json"
chat_model = build_openai_client_chat()
questions_path = "outputs_questions/article_questions02.json"
gen_query(chunks_path, chat_model, questions_path)

len(articles) = 5
processing 1,8-萘酰亚胺类有机小分子电致发光材料的研究进展_王启_llm_correct.md


Chunking:  11%|█         | 1/9 [00:24<03:16, 24.57s/file, chunks=32]

processing 1,8-萘酰亚胺类有机小分子电致发光材料的研究进展_王启_llm_correct.md questions 3
Questions saved to outputs_questions/article_questions02.json


Chunking:  22%|██▏       | 2/9 [00:25<01:13, 10.54s/file, chunks=32]

processing 1,8-萘酰亚胺类有机小分子电致发光材料的研究进展_王启_llm_correct.md questions 3
Questions saved to outputs_questions/article_questions02.json


...

processing 12.7cm彩色AM-OLED显示屏的驱动模块_张繁_llm_correct.md questions 2
Questions saved to outputs_questions/article_questions02.json


Chunking:  86%|████████▌ | 6/7 [01:10<00:10, 10.64s/file, chunks=24]

processing 12.7cm彩色AM-OLED显示屏的驱动模块_张繁_llm_correct.md questions 2
Questions saved to outputs_questions/article_questions02.json


Chunking: 100%|██████████| 7/7 [01:18<00:00, 11.23s/file, chunks=24]

processing 12.7cm彩色AM-OLED显示屏的驱动模块_张繁_llm_correct.md questions 2
Questions saved to outputs_questions/article_questions02.json
done 12.7cm彩色AM-OLED显示屏的驱动模块_张繁_llm_correct.md questions.


In [4]:
num_chunks = 5 // 4
num_chunks

1

# answer generation

In [1]:
from utils.answer_generation import build_openai_client_chat, gen_answer
chat_model = build_openai_client_chat()
questions_path = "outputs_questions/article_questions02.json"
answers_path = "outputs_answers/article_answers01.json"
gen_answer(questions_path, chat_model, answers_path)

Answering:   5%|▍         | 1/22 [00:34<12:02, 34.40s/file]

Questions saved to outputs_answers/article_answers01.json


Answering:   9%|▉         | 2/22 [00:35<04:55, 14.78s/file]

Questions saved to outputs_answers/article_answers01.json


Answering:  14%|█▎        | 3/22 [01:08<07:22, 23.29s/file]

...

Questions saved to outputs_answers/article_answers01.json


Answering:  93%|█████████▎| 14/15 [06:15<00:19, 19.69s/file]

Questions saved to outputs_answers/article_answers01.json


Answering: 100%|██████████| 15/15 [06:29<00:00, 25.94s/file]

Questions saved to outputs_answers/article_answers01.json
done 12.7cm彩色AM-OLED显示屏的驱动模块_张繁_llm_correct.md answers.


In [ ]:
import json
def load_questions(questions_path):
    with open(questions_path, "r", encoding="utf-8") as f:
        articles = json.load(f)
        return articles

In [ ]:
articles_questions = load_questions(chunks_path)

In [ ]:
prompt_templates = {
    "gpt": """
        Question: {question}\nContext: {context}\n
        Answer this question using the information given in the context above. Here is things to pay attention to: 
        - First provide step-by-step reasoning on how to answer the question. 
        - In the reasoning, if you need to copy paste some sentences from the context, include them in ##begin_quote## and ##end_quote##. This would mean that things outside of ##begin_quote## and ##end_quote## are not directly copy paste from the context. 
        - End your response with final answer in the form <ANSWER>: $answer, the answer should be succinct.
        You MUST begin your final answer with the tag "<ANSWER>:".
    """,
    "llama": """
        Question: {question}
        Context: {context}

        Answer this question using the information given in the context above.
        
        Instructions:
        - Provide step-by-step reasoning on how to answer the question.
        - Explain which parts of the context are meaningful and why.
        - Copy paste the relevant sentences from the context in ##begin_quote## and ##end_quote##.
        - Provide a summary of how you reached your answer.
        - End your response with the final answer in the form <ANSWER>: $answer, the answer should be succinct.
        - You MUST begin your final answer with the tag "<ANSWER>:".

        Here are some samples:

        Example question: What movement did the arrest of Jack Weinberg in Sproul Plaza give rise to?
        Example answer: To answer the question, we need to identify the movement that was sparked by the arrest of Jack Weinberg in Sproul Plaza. 
        The context provided gives us the necessary information to determine this.
        First, we look for the part of the context that directly mentions Jack Weinberg's arrest. 
        We find it in the sentence: ##begin_quote##The arrest in Sproul Plaza of Jack Weinberg, a recent Berkeley alumnus and chair of Campus CORE, 
        prompted a series of student-led acts of formal remonstrance and civil disobedience that ultimately gave rise to the Free Speech Movement##end_quote##.
        From this sentence, we understand that the arrest of Jack Weinberg led to student-led acts which then gave rise to a specific movement. 
        The name of the movement is explicitly mentioned in the same sentence as the "Free Speech Movement."
        Therefore, based on the context provided, we can conclude that the arrest of Jack Weinberg in Sproul Plaza gave rise to the Free Speech Movement.
        <ANSWER>: Free Speech Movement
    """,
    "deepseek": """
        Question: {question}\nContext: {context}\n
        使用上述给定的上下文，回答问题。注意：
        - 首先，请提供有关如何回答问题的详细 reasoning。
        - 在 reasoning 中，如果需要复制上下文中的某些句子，请将其包含在 ##begin_quote## 和 ##end_quote## 中。 这意味着 ##begin_quote## 和 ##end_quote## 之外的内容不是直接从上下文中复制的。
        - 结束你的回答，以 final answer 的形式 <ANSWER>: $answer，答案应该简洁。
        你必须以<Reasoning>: 开头，包含 reasoning 相关的内容；以 <ANSWER>: 开头，包含答案。
    """,
    "deepseek-v2": """{
        "instruction":"你是一个半导体显示领域的资深专家，你掌握TFT、OLED、LCD、QLED、EE、Design等显示半导体显示领域内的相关知识。请根据输入中的切片信息和问题进行回答。切片信息是可能相关的资料，切片信息的内容庞杂，不一定会包含目标答案，请仔细阅读每个切片后再作答，不得出现错误。",
        "input": {
            "context": "{context}",
            "question": "{question}"
        },
        "output": {
            "answer": "根据切片中提供的有效信息对问题进行详尽的回答，推荐分点回答格式。"
        },
        "requirements": {
            "criteria": "根据提供的切片信息提取有效信息进行回答",
            "format": "输出内容必须用中文作答。"
        }
    }"""
}

In [ ]:
def gen_answer_prompt(question: str, chunk4: list[dict], prompt_key : str = "gpt") -> list[str]:
    """
    Encode multiple prompt instructions into a single string for the general case (`pdf`, `json`, or `txt`).
    """
    
    messages = []
    chunkstr = get_chunkstr(chunk4)
    prompt = prompt_templates[prompt_key].replace("{question}", question).replace("{context}", chunkstr)
    messages.append({"role": "system", "content": "You are a helpful question answerer who can provide an answer given a question and relevant context."})
    messages.append({"role": "user", "content": prompt})
    return messages

In [ ]:
questions_path = "outputs_answers/article_questions01.json"
answers_path = "outputs_answers/article_answers01.json"
def gen_answer(questions_path, chat_model, answers_path):
    articles = load_questions(questions_path)
    for a_name,question_dicts in articles.items():
        futures = []
        with tqdm(total=num_chunks, desc="Chunking", unit="file") as pbar:
            with ThreadPoolExecutor(max_workers=2) as executor:
                for question_dict in question_dicts:
                    chunk4 = question_dict["oracle_chunks"]
                    futures.append(executor.submit(generate_label, chat_model, chunk4, x=2, model=" deepseek-r1-250120", prompt_key="deepseek-v2"))
                for future in as_completed(futures):
                    answers = future.result()
                    save_answers(answers, a_name, answers_path)
                print(f"done {a_name} questions.")

 
def generate_label(chat_completer: ChatCompleter, question: str, chunk4: list[dict], doctype: DocType = "md", model: str = None, prompt_key : str = "deepseek-v2") -> str | None:
    """
    Generates the label / answer to `question` using `context` and deepseek.
    """
    messages = gen_answer_prompt(question, chunk4, prompt_key)
    response = chat_completer(
        model=model,
        messages=messages,
        n=1,
        temperature=0,
        max_tokens=2048,
    )
    reasoning_content = response.choices[0].message.reasoning_content
    response = response.choices[0].message.content
    return response, reasoning_content
                

# data synthesis

In [ ]:
from utils.data_synthesis import syn_data

chunks_path = "outputs_chunks/article_chunks02.json"
answers_path = "outputs_answers/article_answers01.json"
syndatas_path = "outputs_syndatas/syndatas03.json"
syn_data(chunks_path, answers_path, syndatas_path, num_distract=4)

Syndatas saved to outputs_syndatas/syndatas03.json
Syndatas saved to outputs_syndatas/syndatas03_instruction.json
done 1,8-萘酰亚胺类有机小分子电致发光材料的研究进展_王启_llm_correct.md syndata.
Syndatas saved to outputs_syndatas/syndatas03.json
Syndatas saved to outputs_syndatas/syndatas03_instruction.json
done 10万象素a-Si_TFT有源矩阵的优化技术_徐重阳_llm_correct.md syndata.
Syndatas saved to outputs_syndatas/syndatas03.json
Syndatas saved to outputs_syndatas/syndatas03_instruction.json
done 10大新显示技术_llm_correct.md syndata.
Syndatas saved to outputs_syndatas/syndatas03.json
Syndatas saved to outputs_syndatas/syndatas03_instruction.json
done 12.7cm彩色AM-OLED显示器分场驱动研究_沈亮_llm_correct.md syndata.
Syndatas saved to outputs_syndatas/syndatas03.json
Syndatas saved to outputs_syndatas/syndatas03_instruction.json
done 12.7cm彩色AM-OLED显示屏的驱动模块_张繁_llm_correct.md syndata.


In [ ]:
# import uuid
import os
import json
import random
from utils.common_utils import load_articles

def save_syndatas(datasyn_list, filename):
    # 判断 filename 是否存在，如果存在则追加写入，否则创建新文件
    if os.path.exists(filename):
        with open(filename, 'r', encoding="utf-8") as f:
            existing = json.load(f)
        existing.extend(datasyn_list)
        with open(filename, 'w', encoding="utf-8") as f:
            json.dump(existing, f, ensure_ascii=False, indent=4)
    else:
        with open(filename, 'w', encoding="utf-8") as f:
            json.dump(datasyn_list, f, ensure_ascii=False, indent=4)
    print(f"Syndatas saved to {filename}")

def syn_data(chunks_path, answers_path, syndatas_path, num_distract: int = 3):
    articles_chunks = load_articles(chunks_path)
    all_chunks = {}
    for a_name,chunks in articles_chunks.items():
        for chunk in chunks:
            all_chunks[chunk["chunk_id"]] = chunk
    articles_answers = load_articles(answers_path)
    for a_name,qa_dicts in articles_answers.items():
        datasyn_list = []
        for qa_dict in qa_dicts:
            # datasyn = {
            #     # "id": None,
            #     "type": None,
            #     "question": None,
            #     "noisy_chunks": None,
            #     "oracle_chunks": None,
            #     "reasoning_answer": None
            # }
            chunk4 = qa_dict["oracle_chunks"]
            question = qa_dict["question"]
            reasoning_answer = qa_dict["reasoning_answer"]
            # datasyn["id"] = str(uuid.uuid4())
            # datasyn["type"] = "general"
            # datasyn["question"] = question
            # datasyn["oracle_chunks"] = chunk4
            # datasyn["reasoning_answer"] = reasoning_answer
            
            docs = [chunk.copy() for chunk in qa_dict["oracle_chunks"]]
            indices = list(all_chunks.keys())
            for chunk in chunk4:
                if chunk["chunk_id"] in indices:
                    indices.remove(chunk["chunk_id"])
            for chunk_id in random.sample(indices, num_distract):
                docs.append(all_chunks[chunk_id])
            random.shuffle(docs)
            # datasyn["noisy_chunks"] = docs
            # datasyn_list.append(datasyn)
            
            datasyn1 = {
                "instruction": "",
                "input": "",
                "output": ""
            }
            instruction_prompt = """{
                "instruction":"你是一个半导体显示领域的资深专家，你掌握TFT、OLED、LCD、QLED、EE、Design等显示半导体显示领域内的相关知识。请根据输入中的切片信息和问题进行回答。切片信息是可能相关的资料，切片信息的内容庞杂，不一定会包含目标答案，请仔细阅读每个切片后再作答，不得出现错误。",
                "input": {
                    "context": "{context}",
                    "question": "{question}"
                },
                "output": {
                    "answer": "根据切片中提供的有效信息对问题进行详尽的回答，推荐分点回答格式。"
                },
                "requirements": {
                    "criteria": "根据提供的切片信息提取有效信息进行回答",
                    "format": "输出内容必须用中文作答。"
                }
            }"""
            ichunkstr = get_chunkstr(docs)
            instruction = instruction_prompt.replace("{context}", ichunkstr).replace("{question}", question)
            datasyn1["instruction"] = instruction
            datasyn1["output"] = reasoning_answer
            datasyn_list.append(datasyn1)
            
        
        save_syndatas(datasyn_list, syndatas_path)
        print(f"done {a_name} syndata.")



In [19]:
chunks_path = "outputs_chunks/article_chunks02.json"
answers_path = "outputs_answers/article_answers01.json"
syndatas_path = "outputs_syndatas/syndatas02.json"
syn_data(chunks_path, answers_path, syndatas_path, num_distract=4)

Syndatas saved to outputs_syndatas/syndatas02.json
done 1,8-萘酰亚胺类有机小分子电致发光材料的研究进展_王启_llm_correct.md syndata.
Syndatas saved to outputs_syndatas/syndatas02.json
done 10万象素a-Si_TFT有源矩阵的优化技术_徐重阳_llm_correct.md syndata.
Syndatas saved to outputs_syndatas/syndatas02.json
done 10大新显示技术_llm_correct.md syndata.
Syndatas saved to outputs_syndatas/syndatas02.json
done 12.7cm彩色AM-OLED显示器分场驱动研究_沈亮_llm_correct.md syndata.
Syndatas saved to outputs_syndatas/syndatas02.json
done 12.7cm彩色AM-OLED显示屏的驱动模块_张繁_llm_correct.md syndata.


In [14]:
import random
d1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
random.shuffle(d1)
d1


[3, 10, 9, 2, 4, 6, 5, 8, 1, 7]